In [ ]:
from google.colab import files
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# 파일 업로드
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
input_format = os.path.splitext(image_path)[1][1:].upper()

if input_format == 'JPG':
    input_format = 'JPEG'

original_image = Image.open(image_path).convert('RGB')

# Gaussian 노이즈 추가 함수
def add_gaussian_noise(image, intensity=0.1):
    np_image = np.array(image, dtype=np.float32)
    mean = 0
    stddev = intensity * 255
    noise = np.random.normal(mean, stddev, np_image.shape)
    noisy_image = np.clip(np_image + noise, 0, 255)
    return Image.fromarray(noisy_image.astype(np.uint8))

# Salt-and-Pepper 노이즈 추가 함수
def add_salt_and_pepper_noise(image, intensity=0.05):
    np_image = np.array(image, dtype=np.uint8)
    prob = intensity
    noisy_image = np_image.copy()

    num_salt = np.ceil(prob * np_image.size * 0.5)
    num_pepper = np.ceil(prob * np_image.size * 0.5)

    coords_salt = [np.random.randint(0, i, int(num_salt)) for i in np_image.shape[:2]]
    noisy_image[coords_salt[0], coords_salt[1], :] = 255

    coords_pepper = [np.random.randint(0, i, int(num_pepper)) for i in np_image.shape[:2]]
    noisy_image[coords_pepper[0], coords_pepper[1], :] = 0

    return Image.fromarray(noisy_image)

# Correlated Noise 추가 함수
def add_correlated_noise(image, intensity=0.1):
    np_image = np.array(image, dtype=np.float32)
    mean = 0
    stddev = intensity * 255
    noise = np.random.normal(mean, stddev, np_image.shape[:2])

    kernel = np.array([[0.1, 0.4, 0.1], [0.4, 1.0, 0.4], [0.1, 0.4, 0.1]])
    correlated_noise = np.zeros_like(noise)
    padded_noise = np.pad(noise, 1, mode='edge')

    for i in range(correlated_noise.shape[0]):
        for j in range(correlated_noise.shape[1]):
            correlated_noise[i, j] = np.sum(padded_noise[i:i+3, j:j+3] * kernel)

    correlated_noise = np.repeat(correlated_noise[:, :, np.newaxis], 3, axis=2)
    noisy_image = np.clip(np_image + correlated_noise, 0, 255)
    return Image.fromarray(noisy_image.astype(np.uint8))

# Striped Noise 추가 함수
def add_striped_noise_fixed(image, stripe_width=20, intensity=10):
    np_image = np.array(image, dtype=np.int32)
    height, width, channels = np_image.shape

    stripe_pattern = np.zeros((height, width), dtype=np.int32)
    for i in range(0, height, stripe_width * 2):
        stripe_pattern[i:i + stripe_width, :] = intensity

    striped_image = np_image + stripe_pattern[:, :, None]
    striped_image = np.clip(striped_image, 0, 255)
    return Image.fromarray(striped_image.astype(np.uint8))

# 노이즈 방식을 선택하는 라디오 버튼
image_choice_widget = widgets.RadioButtons(
    options=[
        '1. Gaussian Noise',
        '2. Salt-and-Pepper Noise',
        '3. Correlated Noise',
        '4. Striped Noise',
        '5. Gaussian + Salt-and-Pepper'
    ],
    description='노이즈 방식:',
    value=None
)

# Gaussian 노이즈 노이즈 강도 슬라이더
gaussian_slider = widgets.FloatSlider(
    value=0.1, min=0.05, max=0.4, step=0.01, description='Gaussian:', continuous_update=False
)

# Salt-and-Pepper 노이즈 강도 슬라이더
sap_slider = widgets.FloatSlider(
    value=0.03, min=0.01, max=0.2, step=0.01, description='Salt&Pepper:', continuous_update=False
)

# Correlated 노이즈 강도 슬라이더
correlated_slider = widgets.FloatSlider(
    value=0.05, min=0.01, max=0.1, step=0.01, description='Correlated:', continuous_update=False
)

# Striped Noise 너비 슬라이더
stripe_width_slider = widgets.IntSlider(
    value=15, min=5, max=50, step=5, description='빗금 간격:', continuous_update=False
)

# Striped Noise 강도 슬라이더
stripe_intensity_slider = widgets.IntSlider(
    value=40, min=10, max=100, step=10, description='빗금 강도:', continuous_update=False
)

# 강도 조절 라벨
intensity_label = widgets.Label(value="노이즈의 강도 및 간격을 조절하세요", visible=True)

# 로딩 라벨
loading_label = widgets.Label(value="노이즈를 추가하는 중입니다. 파일 크기가 클수록 오래 걸릴 수 있습니다. (10분 이상 소요될 수도 있음)", visible=False)

save_option_widget = widgets.RadioButtons(
    options=['Y', 'N'], description='저장 여부:', value=None, disabled=True
)

# 노이즈 설정 버튼
apply_noise_button = widgets.Button(description="노이즈 설정", button_style='info')

def apply_noise(b):
    loading_label.visible = True
    display(loading_label)

    gaussian_intensity = gaussian_slider.value
    sap_intensity = sap_slider.value
    correlated_intensity = correlated_slider.value
    stripe_width = stripe_width_slider.value
    stripe_intensity = stripe_intensity_slider.value

    # 노이즈 이미지 생성
    global gaussian_noisy_image, salt_and_pepper_noisy_image, correlated_noisy_image
    global combined_gaussian_sap, striped_noisy_image

    gaussian_noisy_image = add_gaussian_noise(original_image, intensity=gaussian_intensity)
    salt_and_pepper_noisy_image = add_salt_and_pepper_noise(original_image, intensity=sap_intensity)
    correlated_noisy_image = add_correlated_noise(original_image, intensity=correlated_intensity)
    combined_gaussian_sap = add_salt_and_pepper_noise(gaussian_noisy_image, intensity=sap_intensity)
    striped_noisy_image = add_striped_noise_fixed(original_image, stripe_width, stripe_intensity)

    # 이미지 출력
    fig, axes = plt.subplots(2, 3, figsize=(20, 8))
    fig.subplots_adjust(hspace=0.2, wspace=0.03)

    images = [
        (original_image, "Original Image"),
        (gaussian_noisy_image, f"1. Gaussian Noise ({gaussian_intensity})"),
        (salt_and_pepper_noisy_image, f"2. Salt-and-Pepper Noise ({sap_intensity})"),
        (correlated_noisy_image, f"3. Correlated Noise ({correlated_intensity})"),
        (striped_noisy_image, f"4. Striped Noise (Width: {stripe_width}, Intensity: {stripe_intensity})"),
        (combined_gaussian_sap, "5. Gaussian + Salt-and-Pepper")
    ]

    for ax, (img, title) in zip(axes.flatten(), images):
        ax.imshow(img, aspect="equal")
        ax.axis("off")
        ax.set_title(title, fontsize=12, pad=10, loc='center')
        ax.set_xlim(-0.5, img.size[0] - 0.5)
        ax.set_ylim(img.size[1] - 0.5, -0.5)

    plt.show()

    reset_radio_buttons = widgets.RadioButtons(
        options=['Y', 'N'],
        description='재설정 여부:',
        value=None,
        layout=widgets.Layout(width='auto'),
        description_width='initial'
    )

    display(reset_radio_buttons)

    def reset_noise(b):
        if reset_radio_buttons.value == 'Y':
            apply_noise_button.description = "노이즈 재설정"
            display(intensity_label, gaussian_slider, sap_slider, correlated_slider, stripe_width_slider, stripe_intensity_slider, apply_noise_button)
        elif reset_radio_buttons.value == 'N':
            save_option_widget.disabled = False
            display(save_option_widget)

    reset_radio_buttons.observe(reset_noise, names='value')

def save_image():
    global image_choice_widget, save_name_widget
    image_choice_widget = widgets.RadioButtons(
        options=[
            '1. Gaussian Noise',
            '2. Salt-and-Pepper Noise',
            '3. Correlated Noise',
            '4. Striped Noise',
            '5. Gaussian + Salt-and-Pepper'
        ],
        description='노이즈 방식:',
        value=None,
        layout=widgets.Layout(width='max-content')
    )

    save_name_widget = widgets.Text(
        value='', placeholder='파일 이름 입력', description='파일 이름:'
    )
    save_button = widgets.Button(description="저장", button_style='success')

    save_controls = widgets.HBox([save_name_widget, save_button])

    # 저장 버튼 클릭 이벤트
    def on_save_button_click(b):
        save_name = save_name_widget.value.strip()
        if not save_name:
            print("파일 이름을 입력해야 합니다.")
            return

        # 선택된 이미지 저장
        if image_choice_widget.value == '1. Gaussian Noise':
            selected_image = gaussian_noisy_image
        elif image_choice_widget.value == '2. Salt-and-Pepper Noise':
            selected_image = salt_and_pepper_noisy_image
        elif image_choice_widget.value == '3. Correlated Noise':
            selected_image = correlated_noisy_image
        elif image_choice_widget.value == '4. Striped Noise':
            selected_image = striped_noisy_image
        elif image_choice_widget.value == '5. Gaussian + Salt-and-Pepper':
            selected_image = combined_gaussian_sap
        else:
            print("노이즈 방식을 선택해야 합니다.")
            return

        file_path = f"{save_name}.{input_format.lower()}"

        selected_image.save(file_path, format=input_format)
        print(f"{file_path} 파일이 저장되었습니다. \n")
        files.download(file_path)

        save_option_widget.disabled = False
        save_option_widget.value = None
        display(save_option_widget)

    save_button.on_click(on_save_button_click)

    def on_image_choice_change(change):
        if change['new']:
            display(save_controls)

    image_choice_widget.observe(on_image_choice_change, names='value')

    display(image_choice_widget)


# 저장 여부 변경 이벤트 처리
def on_save_option_change(change):
    if change['new'] == 'Y':
        save_image()
    elif change['new'] == 'N':
        print("저장을 종료합니다.")
        save_option_widget.disabled = True
        image_choice_widget.disabled = True
        gaussian_slider.disabled = True
        sap_slider.disabled = True
        apply_noise_button.disabled = True
        save_name_widget.disabled = True

save_option_widget.observe(on_save_option_change, names='value')

# 초기 상태
display(intensity_label, gaussian_slider, sap_slider, correlated_slider, stripe_width_slider, stripe_intensity_slider, apply_noise_button)

# 버튼 클릭 이벤트 핸들러
apply_noise_button.on_click(apply_noise)


In [ ]:
# 임시 작업 공간(/content) 파일 삭제

import os

# /content/ 디렉토리 내의 모든 파일을 삭제
directory_path = '/content/'

files = os.listdir(directory_path)

for file in files:
    file_path = os.path.join(directory_path, file)
    if os.path.isfile(file_path):
        os.remove(file_path)
        print(f"{file_path} 파일이 삭제되었습니다.")